In [1]:
# Run in python console
import nltk; nltk.download('stopwords')

# Run in terminal or command prompt
#python3 -m spacy download en

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\John\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
pip install pyLDAvis

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [4]:
#pip install spacy==2.2.0

In [33]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import nltk
from nltk.stem import WordNetLemmatizer

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [6]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
#import data
df_id = pd.read_csv("dataset/nonprofit.txt", sep = "|", encoding = "cp1252")
df_text = pd.read_csv("dataset/nonprofit_text.txt", sep = "|", encoding = "cp1252")

In [8]:
#merge sets
df_merge = pd.merge(df_id, df_text, how='inner', on='nonprofit_id')

In [9]:
#filter by TN entries
df_text = df_merge.loc[(df_merge["stabbrv"]=='TN')]

In [10]:
# removes the empty descriptions
df_text = df_text[df_text["description"].str.len() > 0]

# combines all by nonprofit_id to a single description
df_text = df_text.groupby(['nonprofit_id'], as_index = False).agg({'description': ' '.join})

In [11]:
df_text = df_text[df_text["description"].str.len() > 10]

In [12]:
df_text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10096 entries, 0 to 10159
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   nonprofit_id  10096 non-null  int64 
 1   description   10096 non-null  object
dtypes: int64(1), object(1)
memory usage: 236.6+ KB


In [13]:
#create sample set
#df_text = df_text.sample(n = 250000)
#df_text.head()

In [14]:
data = df_text.description.values.tolist()

In [15]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence).encode('utf-8'), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
print(data_words[:1])

[['provide', 'sound', 'supervised', 'provide', 'sound', 'supervised', 'recreational', 'sports', 'program', 'for', 'youth', 'of', 'our', 'community', 'in', 'which', 'they', 'receive', 'sports', 'training', 'beneficial', 'healthy', 'recreation', 'football', 'program', 'baseball', 'program', 'powell', 'recreation', 'commission', 'inc']]


In [16]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['provide', 'sound', 'supervised', 'provide', 'sound', 'supervised', 'recreational', 'sports', 'program', 'for', 'youth', 'of', 'our', 'community', 'in', 'which', 'they', 'receive', 'sports', 'training', 'beneficial', 'healthy', 'recreation', 'football', 'program', 'baseball', 'program', 'powell', 'recreation', 'commission', 'inc']


In [49]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if(len(token) > 3)])
    return texts_out

In [50]:
#python3 -m spacy download en

In [51]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

print(data_words_nostops[:1])

[['provide', 'sound', 'supervised', 'provide', 'sound', 'supervised', 'recreational', 'sports', 'program', 'youth', 'community', 'receive', 'sports', 'training', 'beneficial', 'healthy', 'recreation', 'football', 'program', 'baseball', 'program', 'powell', 'recreation', 'commission', 'inc']]


In [52]:
print(data_words_nostops[7])

['jackson', 'area', 'chamber', 'commerce', 'foundation', 'inc']


In [53]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

print(data_words_bigrams[:1])

[['provide', 'sound', 'supervised', 'provide', 'sound', 'supervised', 'recreational', 'sports', 'program', 'youth', 'community', 'receive', 'sports', 'training', 'beneficial', 'healthy', 'recreation', 'football', 'program', 'baseball', 'program', 'powell', 'recreation', 'commission', 'inc']]


In [54]:
print(data_words_bigrams[7])

['jackson', 'area', 'chamber', 'commerce', 'foundation', 'inc']


In [55]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams)

print(data_lemmatized[:1])


[['provide', 'sound', 'supervise', 'provide', 'sound', 'supervise', 'recreational', 'sport', 'program', 'youth', 'community', 'receive', 'sport', 'training', 'beneficial', 'healthy', 'recreation', 'football', 'program', 'baseball', 'program', 'powell', 'recreation', 'commission']]


In [56]:
print(data_lemmatized[7])

['jackson', 'area', 'chamber', 'commerce', 'foundation']


In [57]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:2])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 3), (8, 2), (9, 1), (10, 2), (11, 1), (12, 2), (13, 2), (14, 2), (15, 1), (16, 1)], [(3, 2), (8, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1)]]


In [58]:
# Build LDA model
lda_model = gensim.models.ldamulticore.LdaMulticore(workers = 3, corpus=corpus,
                                           id2word=id2word,
                                           num_topics=12, 
                                           random_state=42,
                                           chunksize=100,
                                           passes=10,
                                           per_word_topics=True)

In [59]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.039*"support" + 0.028*"music" + 0.022*"program" + 0.019*"youth" + '
  '0.017*"activity" + 0.017*"provide" + 0.014*"fund" + 0.013*"event" + '
  '0.013*"veteran" + 0.013*"organization"'),
 (1,
  '0.024*"legal" + 0.021*"prevention" + 0.021*"education" + 0.018*"labor" + '
  '0.018*"treatment" + 0.017*"advocacy" + 0.017*"abuse" + 0.016*"victim" + '
  '0.015*"safety" + 0.015*"agreement"'),
 (2,
  '0.080*"provide" + 0.047*"service" + 0.034*"housing" + 0.024*"home" + '
  '0.024*"family" + 0.023*"individual" + 0.020*"assistance" + '
  '0.014*"organization" + 0.014*"low_income" + 0.011*"support"'),
 (3,
  '0.108*"county" + 0.026*"animal" + 0.024*"rescue" + 0.022*"tennessee" + '
  '0.018*"public" + 0.017*"park" + 0.013*"emergency" + 0.012*"land" + '
  '0.010*"united_way" + 0.009*"shelter"'),
 (4,
  '0.315*"foundation" + 0.046*"family" + 0.023*"house" + 0.023*"charitable" + '
  '0.019*"west" + 0.018*"habitat" + 0.013*"theatre" + 0.011*"diverse" + '
  '0.010*"dance" + 0.010*"rebuild"'),
 

In [60]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.346905469418031

Coherence Score:  0.435584772591944


In [61]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\Users\John\anaconda3\envs\hackMT\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.160946  0.008832       1        1  20.470280
5      0.151864 -0.110847       2        1  17.444168
2      0.103401  0.135760       3        1  12.682650
10     0.115816 -0.180734       4        1  10.005474
11     0.047032  0.154028       5        1   7.337626
8      0.033997  0.146609       6        1   6.450978
0      0.081776  0.033301       7        1   6.305039
6      0.013836 -0.008432       8        1   5.471059
7     -0.056006  0.061350       9        1   4.782487
3     -0.133528 -0.244747      10        1   3.660750
1     -0.165576 -0.106652      11        1   3.114223
4     -0.353558  0.111531      12        1   2.275267, topic_info=            Term         Freq        Total Category  logprob  loglift
48    foundation  2839.000000  2839.000000  Default  30.0000  30.0000
340       school  2990.000000  2990.000000  Default  29.0000  29.0000
298         care  2832.000000  2832.000000  Default  28.0000  28.0000
144       health  2523.000000  2523.000000  Default  27.0000  27.0000
157       county  2395.000000  2395.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
1923      entity    60.524603   190.471442  Topic12  -5.0016   2.6366
1454    children    54.858690   164.301403  Topic12  -5.0999   2.6861
779        build    61.437018   663.133909  Topic12  -4.9866   1.4041
3657      legacy    41.249922   108.605932  Topic12  -5.3850   2.8150
145         hope    42.272782   420.699378  Topic12  -5.3605   1.4853

[721 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
516      10  0.989863   abandon
1484      1  0.088899     abuse
1484     10  0.021167     abuse
1484     11  0.884760     abuse
407       2  0.066004  academic
...     ...       ...       ...
383       7  0.060898     young
383       8  0.079168     young
16        1  0.641915     youth
16        4  0.023075     youth
16        7  0.334942     youth

[1304 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 6, 3, 11, 12, 9, 1, 7, 8, 4, 2, 5])

In [62]:
def get_readable_topic(input_doc):
    if len(input_doc) > 1:
        doc = lda_model.id2word.doc2bow(input_doc)
        doc_topics, word_topics, phi_values = lda_model.get_document_topics(doc, per_word_topics=True)
        return lda_model.id2word[word_topics[0][0]]
    else:
        return None

index = 0
for doc in data_lemmatized:
    df_text.loc[index,"topic_model_result"] = get_readable_topic(doc)
    index = index + 1

In [63]:
df_text.head(10)

,nonprofit_id,description,topic_model_result
0,602.0,"Provide sound, supervised PROVIDE SOUND, SUPER...",baseball
1,640.0,"PROVIDE FOOD, CLOTHNG AND FURNITURE FOR ECONOM...",community
3,710.0,MCU MEDICAL FOUNDATION INC Provide financial a...,community
4,778.0,TO PROMOTE EDUCATIONAL OPPORTUNITIES FOR WOMEN...,provide
5,1071.0,PROVIDE A SUPPORTIVE LIVING SERVICES HOME AND ...,youth
6,1134.0,RELIGIOUS YOUTH CAMP GROUP RETREATS AND SUMMER...,community
7,1567.0,TO PROVIDE ASSISTANCE TO CANCER PATIENTS IN NE...,area
8,1593.0,JACKSON AREA CHAMBER OF COMMERCE FOUNDATION INC,black
9,1667.0,WILSON COUNTY BLACK HISTORY COMMITTEE,provide
10,1739.0,PROMOTE AND ENCOURAGE THE SHOWING OF REINING H...,community


In [32]:
df_text.to_csv('output.csv')

In [64]:
df_text["topic_model_result"].nunique()

617